In this notebook, we demonstrate how to slice an stl-file into a gwl file including 3D stitching. The main difficulty so far with slicing STL files is that I have not found an easy way to slice it block by block. I need to create a stack for the full figure first, possibly with a lower resolution (for reducing the storage requirements). After this lowres stack is made, I can then make small sub-blocks using the low-res version to guide me in the assignement of where the material is located. 

In this notebook, we also demonstrate how to do the block ordering when the object spans multiple Z-ranges. If Concorde is not available this could be done with the nearest neighbour code as well, but I have not implemented it yet.

There is one big issue with the resulting GWL-file though: it does not print correctly! Describe puts the higher blocks at the correct height, but when writing with Nanoscribe, everything is printed in the same plane.. => Need to debug this!

In [ ]:
%pylab

In [ ]:
import TipSlicer
import os
import scipy.ndimage as im
from importlib import reload
import tables

In [ ]:
# load STL mesh and get X,Y, Z range
stlName='StanfordDragonBinary.stl'
rootName='StanfordDragon'

lowresSlicing=1
lowresHashing=1

mesh,Zmins,Zmaxs=TipSlicer.loadStl(stlName,centred=False,mulDimension=100)
X,Y,Z=TipSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} -> {}  Y: {} -> {}  Z: {} -> {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)



In [ ]:
Zmins.max(),Zmaxs.max(),Zmins.min()

In [ ]:
# Do a first low resolution full scale stack with a correct infill
lowresH5=rootName+'_lowres.h5'
TipSlicer.stlToStack(mesh,lowresH5,X,Y,Z,Zmins,Zmaxs)

In [ ]:
FOVrad=150  # more or less standard value but should be input
FOVz=300
overlapZ=2

amZ=int((Z.max()-Z.min())/FOVz)+1
Zrange=(Z.max()-Z.min())/amZ
print(amZ,Zrange)

# loop over different Z-slices
centreCoords=[]
for k in range(amZ):
    Zstart=k*Zrange
    Zstop=(k+1)*Zrange+overlapZ

    # now get low-resolution projection figures for this Z-range
    Ground,X,Y=TipSlicer.getProjection(lowresH5,Zstart,Zstop)

    # doOptimalStitching or hexagonal/square and find blockcentres
    optPoints=TipSlicer.getOptimalStitchCentres(Ground,FOVrad,FOVrad,X,Y)

    centreCoords.append(np.column_stack((optPoints,np.ones(len(optPoints))*Zstart)))


In [ ]:
Zgrower=1000
newPPlist=[]
for cc in centreCoords:
    newPPlist.extend(cc*np.array([1,1,Zgrower]))
newPP=np.asarray(newPPlist)

TipSlicer.writeTSPfile('blocks.tsp',newPP)
!concorde -o 'tsp.sol' 'blocks.tsp'
Ord=TipSlicer.readTSPfile('tsp.sol')
ord3DCentres=(newPP[Ord]*np.array([1,1,1/Zgrower]))

if (ord3DCentres[0,2]>ord3DCentres[-1,2]):
    ord3DCentres=ord3DCentres[::-1]

In [ ]:
ord3DCentres.shape

In [ ]:
clf()
scatter(ord3DCentres[:,0],ord3DCentres[:,1],s=ord3DCentres[:,2])
plot(ord3DCentres[:,0],ord3DCentres[:,1],'r-')

In [ ]:
clf()
#scatter(ord3DCentres[:,0],ord3DCentres[:,1],s=ord3DCentres[:,2])
plot(ord3DCentres[:,0],ord3DCentres[:,2],'r.-')

# now continue creating blocks

In [ ]:
zstacks=np.unique(ord3DCentres[:,2])
zstacks

In [ ]:
stackI=np.argwhere(np.abs(ord3DCentres[:,2]-243)<1)[:,0]
optPoints=ord3DCentres[stackI,:2]

In [ ]:
optPoints=ord3DCentres[stackI,:2]
optPoints.shape

In [ ]:
rootName='StanfordDragonNew'

In [ ]:
# write the stacks for the different stitching blocks, will convert these to gwl later.

slicing=1

hashing=1
overlapXY_multiplier=3 # Final overlap will be 2 times this multiplied with hashing

if (not os.path.exists(rootName)):
    os.mkdir(rootName)
genBlockName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.h5'
genGwlName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.gwl'
genRegex=rootName+'/.+gwl'
lowresH5='StanfordDragon_lowres.h5'


# now creates the stacks
zstacks=np.unique(ord3DCentres[:,2])



for k in np.arange(len(zstacks)):
    Zstart=zstacks[k]
    Zstop=Zstart+Zrange
    zz=np.arange(Zstart,Zstop+slicing,slicing)
    sliceIndList= np.arange(len(zz))# adapt for variable slicing

    stackI=np.argwhere(np.abs(ord3DCentres[:,2]-Zstart)<1)[:,0]
    optPoints=ord3DCentres[stackI,:2]
    # generate voronoi polygons
    regions, vertices = TipSlicer.makeVoronoiPolygons(optPoints)

    # do loop over blocks and write out the stacks (needs stl and low-res stack) 
    #                                        => CAN I USE formulaToStack and give stl-name and low-res stack as parameters??
    fullDist=[]
    zCentres=[]
    for i in np.arange(len(optPoints)):
#    for i in np.arange(2):
        # now make coordinate systems
        print('Writing block {} in layer {}'.format(i,k))
        xc=optPoints[i,0]
        yc=optPoints[i,1]

        polygon = vertices[regions[i]]

        am=np.int(np.round(2*FOVrad/hashing))+1
        xx=np.linspace(xc-FOVrad,xc+FOVrad,am)
        yy=np.linspace(yc-FOVrad,yc+FOVrad,am)
        Xloc,Yloc=np.meshgrid(xx,yy)

        MM=TipSlicer.makeVoronoiMask(polygon,Xloc,Yloc)
        # if you need overlap, do binary_dilation on the mask
        MM=im.binary_dilation(MM,iterations=overlapXY_multiplier) # overlap = 2*iterations*hatching

        # with these coordinate systems, apply stlToStack
        blockName=genBlockName.format(k,i)

        TipSlicer.stlToStack(mesh,blockName,Xloc,Yloc,zz[sliceIndList],Zmins,Zmaxs,inFillMaster=lowresH5,writingMask=MM)

In [ ]:
#now convert all hdf5-stacks to gwl-files.

h5Regex=rootName+'/.+\d\.h5' # should end in a digit, so as to avoid taking the _col.h5 files..
                             # alternatively, I could take them, and just reject upon inspection of the h5 file..
fullGwlName=rootName+'.gwl'

amShells=0
ScaffStep=2 # for solid infill => 2*hatching is spacing here
hatchStep=1 # for contour distance and top-bottom infill, not used if amShells==1 and doTopBottom=False

doColourH5=False


fp=os.path.split(h5Regex)
if (fp[0]==''):
    fnames,fnumbs=TipSlicer.findFiles(h5Regex)
else:
    fnames,fnumbs=TipSlicer.findFiles(fp[1],fp[0])

for ff in fnames:
    # should decide on the sliceIndList here. It could differ from the stack, but this is difficult to determine here
    # especially if it needs to be in Z => which Z's are we doing here? Unknown in advance.
    gwlName=ff[:-3]+'.gwl'
    print('Writing gwl: {}'.format(gwlName))
    dists=TipSlicer.stackToGwl(ff,gwlName,amShells,ScaffStep,hatchStep
                               ,atAngles=-1,writeHeader=False,writeColourH5=doColourH5,doTopBottom=True)

if (len(dists)>0):
    if (len(fullDist)==0):
        fullDist=dists
    else:
        fullDist=fullDist+dists

# Write out regex-gwl to follow this path.
TipSlicer.writeGwlForRegex(fullGwlName,genRegex)#,CodeSpeeds,CodeIntensities)